In [1]:
import vegetables

/home/smgr/.pyenv/versions/co4/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Development notes
- No registry actually needs to take place if there's a default type2component map or one supplied on creation. Can just collect right out of the gate
- Need connective function (type to collation) and attribute map. Do we need to this with a subclass? If a func is passed in on init, I can type it appropriately I guess `Callable[[type[CO3],str,str|None],dict]`

## Usage
The Mapper's primary job is to associate class hierarchies with database components. This can be done in a few ways:

1. Manually attaching a type reference to a Component
2. Attaching a type reference to a Component's name as registered in a schema
3. Automatically register the CO3 heirarchy to matching schema component names (through transformation)

## Explicit example steps

In [2]:
from co3 import Mapper

vegetable_mapper = Mapper(
    vegetables.Vegetable,
    vegetables.vegetable_schema
)

In [3]:
# not valid; tables need to be wrapped in CO3 Components
'''
vegetable_mapper.attach(
    vegetables.Vegetable,
    vegetables.vegetable_table,
)
'''

'\nvegetable_mapper.attach(\n    vegetables.Vegetable,\n    vegetables.vegetable_table,\n)\n'

In [4]:
# manually attach component
vegetable_mapper.attach(
    vegetables.Tomato,
    vegetables.vegetable_schema.get_component('tomato'),
    coll_groups={
        'aging':   vegetables.vegetable_schema.get_component('tomato_aging_states'),
        'cooking': vegetables.vegetable_schema.get_component('tomato_cooking_states'),
    },
)

In [5]:
# attach by name in schema
vegetable_mapper.attach(
    vegetables.Tomato,
    'tomato',
    coll_groups={
        'aging':   'tomato_aging_states',
        'cooking': 'tomato_cooking_states',
    },
)

In [6]:
# attach entire type hierarchy w/ type->name map
vegetable_mapper.attach_hierarchy(
    # this might make more sense during init
    vegetables.Vegetable,
    lambda x:x.__name__.lower()
)

In [9]:
vegetable_mapper.get_collation_comp(vegetables.Tomato, 'cooking')

In [7]:
vars(vegetable_mapper)

{'co3_root': vegetables.Vegetable,
 'schema': <co3.schemas.SQLSchema at 0x732074224aa0>,
 'collector': <co3.collector.Collector at 0x7320757da120>,
 'composer': <co3.composer.Composer at 0x7320757da9c0>,
 'attribute_comps': {vegetables.Tomato: <co3.components.SQLTable at 0x732074224cb0>},
 'collation_groups': defaultdict(dict,
             {vegetables.Tomato: {'aging': <co3.components.SQLTable at 0x732074224ce0>,
               'cooking': <co3.components.SQLTable at 0x732074224d10>}})}

## Holistic attachment

In [2]:
def attr_name_map(cls):
    return f'{cls.__name__.lower()}'

def coll_name_map(cls, action_group):
    return f'{cls.__name__.lower()}_{action_group}_states'

vegetables.vegetable_mapper.attach_many(
    vegetables.type_list,
    attr_name_map,
    coll_name_map,
)

In [3]:
# create new CO3 descendant
tomato = vegetables.Tomato('t1', 5)

# test a register collation action
tomato.collate('ripe')

{'age': 4}

In [4]:
vegetables.vegetable_mapper.get_attribute_comp(vegetables.Tomato)

<Component (SQLTable)> tomato

In [5]:
vars(vegetables.vegetable_mapper)

{'schema': <co3.schemas.SQLSchema at 0x7ab568224e60>,
 'collector': <co3.collector.Collector at 0x7ab568225190>,
 'composer': <co3.composer.Composer at 0x7ab5682251c0>,
 'attribute_comps': {vegetables.Vegetable: <Component (SQLTable)> vegetable,
  vegetables.Tomato: <Component (SQLTable)> tomato},
 'collation_groups': defaultdict(dict,
             {vegetables.Vegetable: {},
              vegetables.Tomato: {'aging': <Component (SQLTable)> tomato_aging_states,
               'cooking': <Component (SQLTable)> tomato_cooking_states}})}

In [6]:
vegetables.vegetable_mapper.collect(tomato, ['ripe'])

['3bf42abc-8a12-452f-baf6-38a05fc5d420',
 '271b7b84-846e-4d1d-87f6-bcabc90a7b55',
 'f9fc5d16-c5cb-47a7-9eca-7df8a3ba5d10']

In [7]:
vegetables.vegetable_mapper.collector._inserts

defaultdict(<function co3.collector.Collector.__init__.<locals>.<lambda>()>,
            {'3bf42abc-8a12-452f-baf6-38a05fc5d420': (<Component (SQLTable)> vegetable,
              {'name': 't1', 'color': 'red'}),
             '271b7b84-846e-4d1d-87f6-bcabc90a7b55': (<Component (SQLTable)> tomato,
              {'name': 't1', 'radius': 5}),
             'f9fc5d16-c5cb-47a7-9eca-7df8a3ba5d10': (<Component (SQLTable)> tomato_aging_states,
              {'name': 't1', 'state': 'ripe', 'age': 1})})

In [8]:
vegetables.vegetable_mapper.collector.inserts

{<Component (SQLTable)> vegetable: [{'name': 't1', 'color': 'red'}],
 <Component (SQLTable)> tomato: [{'name': 't1', 'radius': 5}],
 <Component (SQLTable)> tomato_aging_states: [{'name': 't1',
   'state': 'ripe',
   'age': 1}]}